<a href="https://colab.research.google.com/github/Kim-TaeKyoung/one/blob/main/0_1_DB_to_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
w!pip uninstall -y pymongo
!pip install "pymongo[srv]"
!pip install dnspython==2.0.0

In [ ]:
%load_ext google.colab.data_table

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#-*-coding:utf-8-*-
import sys
from pymongo import MongoClient
from pymongo.cursor import CursorType
import pandas as pd
import json
import dns
import pymongo
import numpy
import ast


client = MongoClient(host='mongodb+srv://cluster0.me7dv.mongodb.net', username='dalgo2021', password='TWeliRO3zlTexIUr', port=27017)
db = client.device
collection = db.device_data_devicedata

#ABtest_cursor_skip = 185359 - 78023
cursor = collection.find().skip(0)

# Load All data from database
data = []

for i in cursor:
  try:
    tmp_dict = i['raw']
    data.append(json.loads(tmp_dict))
  except StopIteration:
    break

In [ ]:
# This should matches with the total number of data in database
print(len(data))

185359


In [ ]:
#@title Compability Code for newer json format
format = set([])
for val in data:
  for key in val.keys():
    format.add(key)

print(format)

type1 = []
type2 = []

for val in data:
  try:
    if val['fixed']:
      type1.append(val)
  except:
    type2.append(val)  

old_format = {'fixed' : { 'value' : {}, 'errcode' : {} }, 'variables' : { 'value' : {}, 'errcode' : {} }}
type2_to_old = []

for data in type2:
  old_format = {'fixed' : { 'value' : {}, 'errcode' : {} }, 'variables' : { 'value' : {}, 'errcode' : {} }}

  old_format['fixed']['value'] = {
      'client_id' : data['value']['client_id'],
      'ad_id' : data['value']['ad_id'],
      'display_id' : data['value']['display_id'],
      'inventory_id' : data['value']['inventory_id']
  }

  old_format['fixed']['errcode'] = {
      'client_id' : data['errcode']['client_id'],
      'ad_id' : data['errcode']['ad_id'],
      'display_id' : data['errcode']['display_id'],
      'inventory_id' : data['errcode']['inventory_id']
  }

  old_format['variables']['value'] = {
      'timestamp' : data['timestamp'],
      'camera_segment' : data['value']['camera_segment'],
      'geostamp' : data['value']['geostamp'],
      'RHC' : None,
      'ATTHC' : None,
      'IHC' : None,
      'ACTHC' : None,
      'weatherstamp' : data['value']['weatherstamp'],
      'illuminance' : data['value']['illumiance']
  }

  old_format['variables']['errcode'] = {
      'client_id' : -999,
      'ad_id' : -999,
      'display_id' : -999,
      'inventory_id' : -999,
      'timestamp' : -999,
      'camera_segment' : data['errcode']['camera_segment'],
      'geostamp' : data['errcode']['geostamp'],
      'ATTHC' : -999,
      'RHC' : -999,
      'ACTHC' : -999,
      'IHC' : -999,
      'weatherstamp' : data['errcode']['weatherstamp'],
      'illuminance' : data['errcode']['illumiance'],
  }
  type2_to_old.append(old_format)
  

{'variables', 'fixed', 'errcode', 'timestamp', 'value'}


In [ ]:
# Uncomment this if json is written in newer format
new_data = [*type1, *type2_to_old]
data = new_data

In [ ]:
backup_data = data

In [ ]:
data = backup_data

In [ ]:
# Parse data from jsons
rows = []
errcode = 0

for j in data:
  # If ad_id is null, it's start shot
  if j['fixed']['value']['inventory_id'] != 'null' and \
    j['fixed']['value']['ad_id'] != 'null' and \
    j['fixed']['value']['client_id'] != 'null' and \
    j['fixed']['value']['display_id'] != 'null':

    # If camera errcode is not 0, we don't need it 
    if 'camera_segment' in j['variables']['errcode']:
      errcode = j['variables']['errcode']['camera_segment']
    # For camera compability
    elif 'camera_frame' in j['variables']['errcode']:
      errcode = j['variables']['errcode']['camera_frame']
    
    if 'camera_segment' in j['variables']['value']:
      if j['variables']['value']['camera_segment'] == None:
        segment_name = None
      else:
        segment_name = j['variables']['value']['camera_segment']['segment_name']
    # For camera compability
    if 'camera_frame' in j['variables']['value']:
      segment_name = None
    else:
      assert 'Unknown camera segment name'
    
    if 'camera_segment' in j['variables']['value']:
      if j['variables']['value']['camera_segment'] == None:
        minutes = None
      else:
        minutes = j['variables']['value']['camera_segment']['minutes']
    # For camera compability
    if 'camera_frame' in j['variables']['value']:
      minutes = j['variables']['value']['camera_frame']
    else:
      assert 'Unknown camera segment minutes'
    
    timestamp = j['variables']['value']['timestamp']
    if isinstance(j['fixed']['value']['ad_id'], dict):
      ad_id = j['fixed']['value']['ad_id']['video']
      if 'current_position' in j['fixed']['value']['ad_id']:
        current_position = j['fixed']['value']['ad_id']['current_position']
      elif 'current_time' in j['fixed']['value']['ad_id']:
        current_position = j['fixed']['value']['ad_id']['current_time']
      else:
        current_position = None
    else:
      ad_id = j['fixed']['value']['ad_id']
      current_position = None
      time_remaining = None

    geostamp = j['variables']['value']['geostamp']
    if geostamp is not None:
      geostamp = geostamp.split(',')
      if geostamp[-1] != 'null':
        # Speed is in knots. Convert to km/h
        geostamp[-1] = float(geostamp[-1]) * 1.852
    else:
      geostamp = [None, None, None, None]

    weatherstamp = j['variables']['value']['weatherstamp']
    illuminance = j['variables']['value']['illuminance']

    rows.append([segment_name, minutes, timestamp, ad_id, current_position, geostamp[0], geostamp[1], geostamp[2], geostamp[3], weatherstamp, illuminance])

In [ ]:
#@title For Newer json
# Parse data from jsons
rows = []
errcode = 0

def null2none(val: str):
  return None if val == 'null' else val

for j in data:
    d_value = j['value']
    d_errcode = j['errcode']

    if key in d_errcode.keys():
      if not d_errcode[key] == 'null':
        segment_name = d_value['segment_name']
        minutes = d_value['minutes']
        timestamp = j['timestamp']
        ad_id = d_value['ad_id']['video']
        current_position = d_value['ad_id']['current_position']
        geostamp = [null2none(val) for val in d_value['geostamp'].split(',')]
        weatherstamp = d_value['weatherstamp']
        illuminance = d_value['illuminance']

    rows.append([segment_name, minutes, timestamp, ad_id, current_position, geostamp[0], geostamp[1], geostamp[2], geostamp[3], weatherstamp, illuminance])

In [ ]:
# Cross reference, check if there is any error left in data
errors = []
errcode = 0

for d in data:
  if 'camera_segment' in d['variables']['errcode']:
    if d['variables']['errcode']['camera_segment'] == None:
      errcode = -1
    else:
      errcode = d['variables']['errcode']['camera_segment']
  elif 'camera_frame' in d['variables']['errcode']:
    errcode = d['variables']['errcode']['camera_frame']
  if errcode != 0:
    errors.append(d['variables'])

print('Size of Errors : ', len(errors))
print('Data without Errors : ', len(data) - len(errors))
print('Current Data : ', len(rows))

Size of Errors :  5520
Data without Errors :  179839
Current Data :  185359


In [ ]:
rows_date_sorted = {}

for data in rows:
  timestamp = data[2]
  time_dict_key = timestamp.split('-')[0]

  if not time_dict_key in rows_date_sorted:
    rows_date_sorted.update({time_dict_key : []})

  rows_date_sorted[time_dict_key].append(data)

In [ ]:
rows_date_sorted.keys()
df_rows_date_sorted = {}

for date in rows_date_sorted.keys():
  df_rows_date_sorted.update({date : None})

for key in rows_date_sorted.keys():
  rows_date_sorted[key] = pd.DataFrame(rows_date_sorted[key], columns=['segment_name', 'minutes', 'timestamp', 'ad_id', 'current_time', 'lat', 'lon', 'alt', 'speed', 'weatherstamp', 'illuminance'])

In [ ]:
rows_date_sorted.keys()

dict_keys(['20211007', '20211008', '20211009', '20211011', '20211012', '20211013', '20211014', '20211015', '20211016', '20211017', '20211019', '20211022', '20211023'])

In [ ]:
for key in rows_date_sorted.keys():
  print(key)
  for column in rows_date_sorted[key].keys():
    null_check = rows_date_sorted[key][column].isnull().all()
    print((column, null_check))

In [ ]:
rows_date_sorted['20211023']

In [ ]:
for date in rows_date_sorted.keys():
  rows_date_sorted[date].to_csv('/content/drive/MyDrive/Device_Output-All-' + date +'.csv', index=False)

In [ ]:
#@title Possible device SW error, timestamp is not unique.

edf = []
odf = []

for e in errors:
  query = df.loc[df['timestamp'] == e['value']['timestamp']]
  if not query.empty:
    print(query)
    edf.append(query)
    odf.append(e)

pd.DataFrame(odf)

NameError: ignored

In [ ]:
edf = pd.DataFrame(pd.concat(edf))
edf

,segment_name,minutes,timestamp,ad_id,lon,lat,alt,speed,weatherstamp,illuminance
2059,None,0,20211007-120229KST,아이디올시니어버전,None,None,None,None,None,NaN
9682,None,0,20211008-143429KST,아이디올시니어버전,37.494445,127.11570166666667,80.8,1.61124,None,1817.916667
10314,None,0,20211008-151938KST,아이디올시니어버전,37.494295,127.11507,78.0,null,None,785.000000
10445,None,0,20211008-152631KST,아이디올시니어버전,37.49200666666667,127.11641166666666,41.7,10.8342,None,2008.333333
12196,None,54,20211008-162111KST,타코셔틀,37.50017666666667,127.02640166666667,122.6,0.40744,None,1841.666667
12197,None,54,20211008-162113KST,바베큐셔틀,37.50017166666667,127.02640833333334,122.5,0.463,None,1688.750000
12199,None,54,20211008-162116KST,바베큐셔틀,37.50016166666666,127.02642666666667,122.1,0.81488,None,1753.333333
12200,None,54,20211008-162119KST,바베큐셔틀,37.50016333333333,127.02643166666667,121.9,0.48152,None,1673.750000
12206,None,54,20211008-162130KST,아이디올시니어버전,37.500141666666664,127.026455,120.2,2.33352,None,1718.750000
12207,None,54,20211008-162133KST,아이디올시니어버전,37.50013833333333,127.02645666666666,119.5,0.72228,None,1685.000000
